# Intro

This is my progress in the implementation of Hermite mappings for the quadrature representation of cell integrals. See hermite.tm for the math.

The notebook [elements](elements.ipynb) contains some nomenclature, including definitions of: *finite element*, *dof* (and its action on functions), *intermediate representation*.

There are also [Related commits and PRs](elements.ipynb#Related-commits-and-PRs).


# Basic integration of forms

## FFC  form compilation

The function `ffc.compile_form(F)` produces a  c++ header with the code which is later wrapped into a python API for using the form `F`. It contains classes with:

* Finite elements.
* Local to global DOF maps.
* **Cell integral computation:** an implementation of the interface defined in the abstract class `ufc::cell_integral`. This is what interests us here.
* The form itself.

## A simple example

Begin with the easiest:

$$ \int \phi_i \phi_j \ \mathrm{d}x$$

for all basis functions in $V$. Recall that `ufc::cell_integral::tabulate_tensor()` should compute

$$ \int_T \phi_i(x)\ \mathrm{d}x 
 = \int_T \hat{\phi}_i(F^{-1}(x))\ \mathrm{d}x 
 = \int_\hat{T} \hat{\phi}_i(\hat{x}) J\ \mathrm{d}\hat{x} $$

In [ ]:
from __future__ import print_function

from dolfin import *
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline
import ffc

#from ffc.log import add_logfile, set_level, DEBUG
#set_level(DEBUG)
#add_logfile("/tmp/fenics.log")

import nbimporter
from elements import evaluate_shape_functions_reference

parameters['form_compiler']['no-evaluate_basis_derivatives'] = False

# Notice the high degree...
fc_params = {'representation': 'quadrature',
             'quadrature_degree': 7}

mesh = UnitIntervalMesh(1)
V = FunctionSpace(mesh, "Hermite", 3)

u = TrialFunction(V)
v = TestFunction(V)

#a = u.dx(0).dx(0)*v.dx(0).dx(0)*dx
a = u.dx(0)*v.dx(0)*dx
#a = u*v*dx
A = assemble(a, form_compiler_parameters=fc_params)
with open("/tmp/mass_%s1D.h" % V.ufl_element().family(), "w") as fd:
    out = ffc.compile_form(a, "mass", parameters=fc_params)
    fd.write(out[0])

print("FFC:\n%s" % A.array().round(3))

## One cell integral

The following reproduces the computation performed by the code produced with `ffc.compile_form()`:

In [ ]:
# Hermite shape functions on [0,1]
#sh = [lambda x: 1 - 3 * x**2 + 2 * x**3,
#      lambda x: x - 2 * x**2 + x**3,
#      lambda x: + 3 * x**2 - 2 * x**3,
#      lambda x: - x**2 + x**3]
# quadrature points:
#WP = np.array([0.046910077030668, 0.230765344947158, 0.5, 0.769234655052841, 0.953089922969332])
WP = np.array([0.0694318442029737, 0.330009478207572, 0.669990521792428, 0.930568155797026])
# quadrature weights:
#WW = np.array([0.118463442528095, 0.239314335249683, 0.284444444444444, 0.239314335249683, 0.118463442528095])
WW = np.array([0.173927422568727, 0.326072577431273, 0.326072577431273, 0.173927422568727])
# basis functions evaluated at quadrature points:
#FE0 = np.array([f(WP) for f in sh]).T
#FE0 = evaluate_shape_functions_reference(V, WP).T
FE0 = evaluate_shape_functions_mesh(V, WP)[1].T
# Jacobian for a mesh of constant cell size
det = 1./ V.mesh().num_cells()
# Hermite trafo matrix
H = np.array([1., 0., 0., det])

dim = V.element().space_dimension()
M = np.zeros((dim, dim))
if V.ufl_element().family() == 'Lagrange':
    for ip,j,k in np.ndindex(len(WP), M.shape[0], M.shape[1]):
        M[j,k] += FE0[ip][j]*FE0[ip][k]*WW[ip]*det
elif V.ufl_element().family() == 'Hermite':
    for ip,j,k in np.ndindex(len(WP), M.shape[0], M.shape[1]):
        M[j,k] += (((FE0[ip][2*(j/2)+0])*(H[2 * (j % 2) + 0]) 
                    + (FE0[ip][2*(j/2)+1])*(H[2 * (j % 2) + 1]))*
                   ((FE0[ip][2*(k/2)+0])*(H[2 * (k % 2) + 0]) 
                    + (FE0[ip][2*(k/2)+1])*(H[2 * (k % 2) + 1])))*WW[ip]*det

print("FFC-like computation with quadrature:\n %s" % M.round(3))

In `tabulate_tensor()` for the form `u*v`:

    // Values of basis functions at quadrature points.
    static const double FE0[4][4] = \
    {{0.986207088460911, 0.0601249979387161, 0.0137929115390892, -0.00448606527483159},
    {0.745161426118203, 0.148137063413256, 0.254838573881797, -0.0729661590874814},
    {0.254838573881795, 0.0729661590874805, 0.745161426118205, -0.148137063413257},
    {0.0137929115390886, 0.00448606527483117, 0.986207088460911, -0.0601249979387162}};

In `tabulate_tensor()` for the form `u.dx(0)*v.dx(0)`:

    // Values of basis functions at quadrature points.
    static const double FE0_D1[4][4] = \
    {{-0.387666379281288, 0.736734966156382, 0.387666379281288, -0.12440134543767},
    {-1.32661933500443, 0.00668085429021271, 1.32661933500443, -0.333300189294643},
    {-1.32661933500443, -0.333300189294642, 1.32661933500443, 0.00668085429021441},
    {-0.387666379281282, -0.124401345437667, 0.387666379281282, 0.736734966156384}};

Note that some of the entries have been permuted wrt. the assembled matrix, where the ordering is given by the local-to-global dof mapping. We will need to take this into account later when we manually assemble everything. But first we compute the same integral using the trapezoidal rule as a check:

In [ ]:
B = np.zeros_like(M)
xx = np.linspace(0, 1, 100)
#shapes = evaluate_shape_functions_reference(V, xx)
shapes = evaluate_shape_functions_mesh(V, xx)[1]
for i,j in np.ndindex(B.shape):
    B[i,j] = np.trapz(det*shapes[i]*shapes[j], xx)

print("Trapezoidal:\n%s\n" % B.round(3))
print("Trapezoidal - quadrature: %.1f%% error" % (100*np.linalg.norm(B-M, np.inf)/np.linalg.norm(B, np.inf)))

## Mass matrix assembly

The preceding cell integrals need to be assembled into a global mass matrix in order to compare the computations with those done by dolfin. What matters is then the difference between this matrix and the one computed by dolfin:

In [ ]:
Aa = A.array()
Ah = np.zeros_like(Aa)
dm = V.dofmap()
for cell in range(V.mesh().num_cells()):
    l2g = dm.cell_dofs(cell)   # local to global mapping for cell 
    for i,j in np.ndindex(M.shape):
        Ah[l2g[i], l2g[j]] += M[i,j]
print("Manual assembly:\n%s\n" % Ah.round(3))
print("Dolfin's assembly:\n%s\n" % Aa.round(3))
print("Error = %.2f%%" % (100*np.linalg.norm(Aa - Ah, np.inf)/np.linalg.norm(Ah, np.inf)))

If this is correct, since we used the trapezoidal cell integral with high resolution, it must be that our copy of FFC's output code is wrong... (?)

# A more involved example

We now work with the bilinear form

$$ \int \Delta u \Delta v \ \mathrm{d}x$$

In [ ]:
mesh = UnitIntervalMesh(1, 1)
V = FunctionSpace(mesh, "Hermite", 3)
u = TrialFunction(V)
v = TestFunction(V)
a = inner(div(grad(u)), div(grad(v)))*dx
with open("/tmp/laplace.h", "wt") as fd:
    out = ffc.compile_form(a, prefix="la", parameters=fc_params)
    fd.write(out[0])

# Integration of expressions



In [ ]:
grid_sizes = [32, 64, 128, 256]
p_vals = range(1, 4)
results_her = np.zeros(len(grid_sizes))
results_lag = np.zeros((len(grid_sizes), len(p_vals)))

for i, n in enumerate(grid_sizes):
    mesh = UnitSquareMesh(n, n)
    V = FunctionSpace(mesh, "Hermite", 3)
    W = [None] + [FunctionSpace(mesh, "Lagrange", p) for p in p_vals]
    f = Expression("x[0]*x[0]*x[1]*x[1]", degree=3)
    v = project(f, V)
    w = [None] + [project(f, W[p]) for p in p_vals]
    exact = 1./9.
    results_her[i] = assemble(v*dx)
    results_lag[i] = np.array([assemble(w[p]*dx) for p in p_vals])

In [ ]:
pl.figure(figsize=(10, 10))
pl.plot(grid_sizes, np.log(results_her - exact), label="H3")
for p in p_vals:
    pl.plot(grid_sizes, np.log(results_lag[:,p-1] - exact), label="L%d" % p)
pl.title("Log error")
pl.legend()
_ = pl.show()